<a href="https://colab.research.google.com/github/EureXaAI/EurexaBook/blob/main/letter/EurexaBook_A_Z.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 安装 brax
# 安装 Brax: Google 开发的独立物理引擎, 原生用 JAX 写的
!pip install brax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.3/709.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 14.6 MB/s eta 0:00:00


In [2]:
#@title 导入依赖包
#@markdown ### 本代码研究如何用 brax 和 mujoco 搭建最简单的强化学习案例
# JAX Imports:
import jax
import jax.numpy as jnp

In [86]:
#@title 物理模型
xml_model = """
<mujoco>
  <asset>
    <texture type="2d" name="groundplane" builtin="checker" mark="edge" rgb1="0.9375 0.7226 0.04296"
        rgb2="0 0 0" markrgb="0.8 0.8 0.8" width="1000" height="1000"/>
    <material name="groundplane" texture="groundplane" texuniform="true" texrepeat="5 5"
        reflectance="0.2"/>
  </asset>

  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="floor" pos="0 0 -1" size="0 0 .125" type="plane" material="groundplane" conaffinity="15" condim="3"/>
    <body name="letter_A" pos="0 0 0">
      <!-- 固定铰链 -->
      <joint name="swing" type="hinge" axis="1 -1 0" pos="0 0 0"/>
      <!-- 左侧斜杆 -->
      <geom name="left_leg" type="box" pos="-0.15 0 0" size=".025 .025 .36" quat="1.159 0 0.2588 0" rgba="0.9375 0.7226 0.04296 1"/>
      <!-- 右侧斜杆 -->
      <geom name="right_leg" type="box" pos="0.15 0 0" size=".025 .025 .36" quat="1.1659 0 -0.2588 0" rgba="0.9375 0.7226 0.04296 1"/>
      <!-- 横杆 -->
      <geom name="crossbar" type="box" pos="0 0 0" size=".15 .025 .025" rgba="0.9375 0.7226 0.04296 1"/>
    </body>
  </worldbody>
</mujoco>
"""

In [87]:
#@title 初始化模型 { run: "auto" }
# Brax Imports:
from brax.mjx import pipeline
from brax.io import mjcf, html

# Load the MJCF model
sys = mjcf.loads(xml_model)

# Jitting the init and step functions for GPU acceleration
init_fn = jax.jit(pipeline.init)
step_fn = jax.jit(pipeline.step)

# Initializing the state:
state = init_fn(
    sys=sys, q=sys.init_q, qd=jnp.zeros(sys.qd_size()),
)

In [84]:
#@title 执行模拟
num_steps = 100
state_history = []
ctrl = jnp.zeros(sys.act_size())
for i in range(num_steps):
    state = step_fn(sys, state, act=ctrl)
    state_history.append(state)

In [85]:
#@title 可视化
from IPython.display import HTML

HTML(
    html.render(
        sys=sys,
        states=state_history,
        height=500,
        colab=True,
    ),
)